# Machine Learning 2021 class - Akbarov, Sobala

## Title: Determining number of shares of types of news - Machine Learning Classification methods comparison

## Subject: *classification*

## Introduction

The main goal of the project is to perform regression on a large dataset, i.e. a dataset in which (# columns × # rows) is above 1.000.000
and there at least 20 variables of different types (continuous,
categorical); and then compare the performance of different models and methods.

The **main problem analysed** is to explain the number of shares of online news (their popularity) and try to predict future values.

After the dataset description and some statistical description of the response and explanatory variables, we propose the classification models.

#### Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image 
#import pydot
#from pydot import graph_from_dot_data
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
#import plotly.graph_objs as go
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display, Math, Markdown, Latex, display_markdown
from matplotlib.pyplot import figure
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from IPython.display import Markdown, display 
def printmd(string):  
    display(Markdown(string)) 
from sklearn.metrics import confusion_matrix
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
from scipy import stats


## Data

The dataset was sourced from https://archive.ics.uci.edu/ml/datasets/online+news+popularity.

Let's import our data and analyse its structure.

In [2]:
news = pd.read_csv("../input/onlinenewspopularity/OnlineNewsPopularity.csv")
print(news.shape)

The dataset is comprised of 39644 instances and the following 61 attributes:
<br>
1) url,
<br>
2) timedelta,
<br>
3) n_tokens_title,
<br>
4) n_tokens_content
<br>
5) n_unique_tokens,
<br>
6) n_non_stop_words,
<br>
7) n_non_stop_unique_tokens
<br>
8) num_hrefs,
<br>
9) num_self_hrefs,
<br>
10) num_imgs,
<br>
11) num_videos,
<br>
12) average_token_length,
<br>
13) num_keywords,
<br>
14) data_channel_is_lifestyle,
<br>
15) data_channel_is_entertainment,
<br>
16) data_channel_is_bus,
<br>
17) data_channel_is_socmed,
<br>
18) data_channel_is_tech,
<br>
19) data_channel_is_world,
<br>
20) kw_min_min,
<br>
21) kw_max_min,
<br>
22) kw_avg_min,
<br>
23) kw_min_max,
<br>
24) kw_max_max,
<br>
25) kw_avg_max,
<br>
26) kw_min_avg,
<br>
27) kw_max_avg,
<br>
28) kw_avg_avg,
<br>
29) self_reference_min_shares,
<br>
30) self_reference_max_shares,
<br>
31) self_reference_avg_sharess,
<br>
32) weekday_is_monday,
<br>
33) weekday_is_tuesday,
<br>
34) weekday_is_wednesday
<br>
35) weekday_is_thursday,
<br>
36) weekday_is_friday,
<br>
37) weekday_is_saturday,
<br>
38) weekday_is_sunday,
<br>
39) is_weekend,
<br>
40) LDA_00
<br>
41) LDA_01,
<br>
42) LDA_02,
<br>
43) LDA_03,
<br>
44) LDA_04,
<br>
45) global_subjectivity,
<br>
46) global_sentiment_polarity,
<br>
47) global_rate_positive_words,
<br>
48) global_rate_negative_words,
<br>
49) rate_positive_words,
<br>
50) rate_negative_words,
<br>
51) avg_positive_polarity,
<br>
52) min_positive_polarity,
<br>
53) max_positive_polarity,
<br>
54) avg_negative_polarity,
<br>
55) min_negative_polarity,
<br>
56) max_negative_polarity,
<br>
57) title_subjectivity,
<br>
58) title_sentiment_polarity,
<br>
59) abs_title_subjectivity,
<br>
60) abs_title_sentiment_polarity,
<br>
61) shares.

In [3]:
print(news.head(5))

In [4]:
if pd.isnull(news).sum().sum()==0:
    print("There are no missing values to be removed!")
else:
    print("Some missing values need to be removed!")
    news = news.dropna()

In [5]:
news.info()

### According to information from data news, we come to a conclusion about there is work to be done here:
1) remove gaps form head of columns,
2) drop unpredictable variables,
3) reformatting type of channel and days of week columns (The columns consist of 0 and 1. we don't need them that way. We need to create a new columns for the days of the week and separate for the categories and combine them in that columns)
4) finding outliers and depends on the situation, replace or remove
5) select features and drop the ones with the lowest scores

In [6]:
print(news.columns.tolist())

In [7]:
#removing white spaces from variable names
news.columns = news.columns.str.lstrip()

### Data preparation

##### - weekdays

In [8]:
cl = ['weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday', 'weekday_is_sunday']
weekday = [1, 2, 3, 4, 5, 6, 7]
for i in range(0,7):
    news[cl[i]] = news[cl[i]].replace({1.0: weekday[i], 0.0: np.nan})

news['weekdays'] = news[cl].sum(axis=1).astype('int64')

In [9]:
print(news['weekdays'].value_counts())

##### - channel type

In [10]:
cl2 = ['data_channel_is_lifestyle', 'data_channel_is_entertainment',
       'data_channel_is_bus', 'data_channel_is_socmed',
       'data_channel_is_tech', 'data_channel_is_world']
    
news["category"] = news[cl2].idxmax(axis=1)


news['category'] = news['category'].replace('data_channel_is_lifestyle', 1)
news['category'] = news['category'].replace('data_channel_is_entertainment', 2)
news['category'] = news['category'].replace('data_channel_is_bus', 3)
news['category'] = news['category'].replace('data_channel_is_socmed', 4)
news['category'] = news['category'].replace('data_channel_is_tech', 5)
news['category'] = news['category'].replace('data_channel_is_world', 6)


In [11]:
print(news['category'].value_counts())

In [12]:
#drop unnecessary columns
news2 = news.drop(['url','timedelta','data_channel_is_lifestyle', 'data_channel_is_entertainment',
       'data_channel_is_bus', 'data_channel_is_socmed',
       'data_channel_is_tech', 'data_channel_is_world','weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday', 
               'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday', 'weekday_is_sunday','is_weekend'], axis=True)


In [13]:
news2.describe()

All the columns in our data are made up of 39644 variables, our data is perfectly integrated, and there is no NA.Also, if we look at their maximum and minimum values, we'll see that some columns are very balanced, but for others it's opposite

### Visualizating the data 

In [14]:
def plot_histogram(x):
    plt.hist(x, color='gray',alpha=0.5)
    plt.title("Histogram of'{var_name}".format(var_name=x.name))
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.show()

#### Response variable

In [15]:
#boxplot
shares = news2[['shares']]
shares.plot(kind='box', subplots=True, sharex=False, sharey=False, figsize = (20.0, 10.0))
plt.title("Boxplot",size=30,loc='left')
plt.show()

In [16]:
plot_histogram(news2['shares'])

According to the data we got from the plots,it appears Average of shares is between 0 and 200,000 but there is several outliers

#### Explanatory variables

In [17]:
df = news2[news2.columns.difference(['shares'])]
df.plot(kind='box', subplots=True, layout=(16,4), sharex=False, sharey=False, figsize = (20.0, 30.0))
plt.show()

As can be seen on the boxplots, many of our variables have many outliers. We will need to deal with them, before applying any models to our data.

In [18]:
plt.style.use('seaborn-white')
df.hist(figsize = (20.0, 40.0),layout=(16,4), density = 1, color = 'lightblue')
plt.show()

A co-efficient close to 1 means that there’s a very strong positive correlation between the two variables. In our case, the maroon shows very strong correlations. The diagonal line is the correlation of the variables to themselves — so they’ll obviously be 1.

In [19]:
pearsoncorr = news2.corr(method='pearson')

In [20]:
sns.set(style="white")
corr = pearsoncorr
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(40, 40))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, annot = corr.round(2), linewidths=.5, cbar_kws={"shrink": .10})

A co-efficient close to 1 means that there’s a very strong positive correlation between the two variables. In our case, Looking at this we can quickly see that:
we observe mild redness between average,minimum,maximum positive negative words variables. also(polarity and subjectivity variables)
most of correlations are closer to zero even negative 



In [21]:
ax = sns.boxplot(data=news2, orient="h", palette="Set2")

Mutual information is a distance between two probability distributions. Correlation is a linear distance between two random variables. There is no big differences for majority of variables between Mutual Information and Pearson results that means both of them are usefull at same extend, just some exceptions: n_non_stop_words,min_positive_polarity,kw_avg_avg 

In [22]:
from sklearn import feature_selection
expl = news2[news2.columns.difference(['shares'])]
minfos=[]
for var in df:
    print("\n", var)
    print("Pearson", stats.pearsonr(news2["shares"], news2[var]))
    print("Mutual info", feature_selection.mutual_info_classif(news2[var].values.reshape(-1,1),news["shares"].values))
    minfos.append(feature_selection.mutual_info_classif(news2[var].values.reshape(-1,1),news2["shares"].values))

imp = list(zip(minfos, expl))
imp.sort(reverse=True)
imp

All of our variables have low scores. We remove five variables with lowest scores (close to 0).

In [23]:
X = news2.drop(['title_sentiment_polarity','rate_negative_words',
                'n_non_stop_unique_tokens','global_rate_negative_words',
                'average_token_length'],  axis=True)

In [24]:
X.shape

In [25]:
X.head(10)

### Finding outliers

In [26]:
for col_name in X.columns :
    if X[col_name].dtypes == 'int64':
        unique_cat = len(X[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique categories (integer)".format(
        col_name=col_name, unique_cat=unique_cat))
        
for col_name in X.columns :
    if X[col_name].dtypes == 'float64':
        unique_cat = len(X[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique categories (float)".format(
        col_name=col_name, unique_cat=unique_cat))

### -> Tukey's method for finding outliers

In [27]:
def find_outliers_tukey(x):
    q1 = np.percentile(x,25)
    q3 = np.percentile(x,75)
    iqr = q3-q1
    floor = q1- 1.5*iqr
    ceiling = q3+1.5*iqr
    outlier_indices = list(x.index[(x<floor)|(x>ceiling)])
    outlier_values = list(x[outlier_indices])
    return outlier_indices, outlier_values

In [28]:
X2 = X.copy() 
for i in X2.columns :
    tukey_indices, tukey_values = find_outliers_tukey(X2[i])
    tukey_values = np.sort(tukey_values)
    print("Column '{i}' outliers list : {tukey_values} ".format(i=i, tukey_values=tukey_values))
    

In [29]:
X2.describe()

In [30]:
for i in X2.columns :
    a = X2[i].median()
    print("Column '{i}' median is : {a} ".format(i=i, a=a))

In [31]:
for i in X2.columns:
    o_mean = X2[i].mean()
    o_std = X2[i].std()
    
    low = o_mean - (3*o_std)
    high = o_mean + (3*o_std)
    
    outliers = X2[(X2[i] < low) | (X2[i] < high)]
print(outliers.head())

### Dealing with Outliers:

In [32]:
for i in X2.columns:
    y = X2[i]
    removed_outliers = y.between(y.quantile(.05), y.quantile(.95))
    index_names = X2[~removed_outliers].index # INVERT removed_outliers!!
    X2.drop(index_names, inplace=True)
    


In [33]:
X2

In [34]:
ax = sns.boxplot(data=X2, orient="h", palette="Set2", dodge=True, width=4 ,fliersize=5 ,linewidth=0.1)

In [35]:
tech_exp = X2[X2.columns[:10]]
df = tech_exp[tech_exp.columns.difference(['shares'])]
df.plot(kind='box', subplots=True, layout=(5,2), sharex=False, sharey=False, figsize = (20.0, 30.0))
plt.show()

In [36]:
plt.style.use('seaborn-white')
df.hist(figsize = (20.0, 20.0),layout=(5,3), density = 1, color = 'lightblue')
plt.show()

In [37]:
sns.set(style="white")
corr = X2.corr('pearson')
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(30, 30))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, annot = corr.round(2), linewidths=.5, cbar_kws={"shrink": .10})

In [38]:
ax = sns.boxplot(data=X2, orient="h", palette="Set2")

After deleting outliers,although data still not perfectly balanced,but much more balanced than previous one of itself.We have to improve data as far as we can in order to build convenient models later

## Preparation for split data

In [39]:
print(X2['shares'])

In [40]:
X2['shares'].nunique()
X2['shares'].duplicated().sum()

In [41]:
X2['shares'].value_counts()

In [42]:
#countplot
sns.set(style="whitegrid")
fig = plt.figure(figsize = (10,6))
sns.countplot('shares', data=X2, palette='mako').set(title='Countplot',)

In [43]:
A = pd.DataFrame(X2.shares.value_counts())

In [44]:
A.describe()

In [45]:
data=X2.copy()

In [46]:
print(data.shares)

In [47]:
data['shares'].median()

We spilt the categories by median value.

In [48]:
data['shares'] = data['shares'].apply(lambda x: 0 if x <1300 else 1) 

In [49]:
data.shares.value_counts()

Our dataset at this point is well balanced, i.e. it does not require as to do any sampling.

In [50]:
XA = data.copy()

## Train and test data split

#### Standardise the data

In [51]:
XB = XA.loc[:, XA.columns != 'shares']
y= XA[['shares']]

scaler = preprocessing.StandardScaler()
#names = XA.columns
Xt = scaler.fit_transform(XB)
Xt = pd.DataFrame(Xt)

We also prepare a dataset with normalised data for kNN regression.

In [52]:
scaler = MinMaxScaler()
names = XB.columns
Xn = scaler.fit_transform(XB)
Xn = pd.DataFrame(Xn)

In [53]:
test_AUC = {}
train_AUC = {}
test_acc = {}
train_acc = {}

#### Split the data

We chose to split the data into train and test to 80% and 20% of our dataset, respectively.

In [54]:
X_train, X_test, y_train, y_test = train_test_split(Xt , y, test_size=0.2, random_state = 1)
kf = KFold(n_splits = 5, shuffle = True,  random_state=55) ##this is a must - its a validator (he asked about this durin 1st presentation)
print("Xt shape:",Xt.shape)
print("y shape:",y.shape)
print("X train:",X_train.shape,"X test:",X_test.shape)
print("y train:",y_train.shape,"y test:",y_test.shape)

In [55]:
print(y.dtypes)

## Models

### Logistic Regression 

In [56]:
kf = KFold(n_splits = 5, shuffle = True,  random_state=55)

for train, test in kf. split(Xt.index.values):
    X_train = Xt.iloc[train]
    y_train = y.iloc[train]
    X_test = Xt.iloc[test]
    y_test = y.iloc[test]

    reg = LogisticRegression()
    reg.fit(X_train,y_train)

    pred_train = reg.predict(X_train)
    pred = reg.predict(X_test)
    
    accuracy_train = []
    precision_train = []
    ra_train = []
    recall_train = []
    
    accuracy = metrics.accuracy_score(y_train, pred_train)
    accuracy_train.append(accuracy)
    precision = metrics.precision_score(y_train, pred_train)
    precision_train.append(precision)
    ra= metrics.roc_auc_score(y_train, pred_train)
    ra_train.append(ra)
    recall= metrics.recall_score(y_train, pred_train)
    recall_train.append(recall)
    
    accuracy_test = []
    precision_test = []
    ra_test = []
    recall_test = []

    accuracy = metrics.accuracy_score(y_test, pred)
    accuracy_test.append(accuracy)
    precision = metrics.precision_score(y_test, pred)
    precision_test.append(precision)
    ra= metrics.roc_auc_score(y_test, pred)
    ra_test.append(ra)
    recall= metrics.recall_score(y_test, pred)
    recall_test.append(recall)

printmd('**TRAIN**')
print(' Accuracy:',np.mean(accuracy_train),'\n', 'Precision:', np.mean(precision_train), 
      '\n', 'Roc Auc score:', np.mean(ra_train), '\n', 'Recall:',np.mean(recall_train))

printmd('**TEST**')
print(' Accuracy:',np.mean(accuracy_test),'\n', 'Precision:', np.mean(precision_test), 
      '\n', 'Roc Auc score:', np.mean(ra_test), '\n', 'Recall:',np.mean(recall_test))

In [57]:
lr1 = {'TRAIN Accuracy': np.mean(accuracy_train), 'TRAIN Precision': np.mean(precision_train), 
         'TRAIN ROC AUC': np.mean(ra_train), 'TRAIN Recall': np.mean(recall_train)}
lr2 = {'TEST Accuracy': np.mean(accuracy_test), "TEST Precision": np.mean(precision_test), 
        'Test ROC AUC': np.mean(ra_test), 'TEST Recall': np.mean(recall_test)}

The results for precision and accuracy are not very satisfying, however, tehy are close for test data to the train data results. RAS is more than 0.5 which means that there is a quite high chance that the classifier will be able to distinguish the class values. Recall is also above 0.5.

The confusion matrix gives us the false positives and false negatives (which are close in numbers). The model seems to be better at predicting )s.

In [58]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, pred)
plt.figure(figsize = (8,6))
sns.heatmap(confusion_matrix, annot=True, fmt="d")

In [59]:
logit_roc_auc = np.mean(ra_test)
fpr, tpr, thresholds = roc_curve(y_test, reg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [60]:
test_AUC.update( {'Logistic regression': np.mean(ra_test)} )
train_AUC.update( {'Logistic regression': np.mean(ra_train)} )
test_acc.update( {'Logistic regression': np.mean(accuracy_test)} )
train_acc.update( {'Logistic regression': np.mean(accuracy_train)} )

The bigger the area under the curve the better. However, the result is difficult to interpret due to unclear chnages in distances between the curves. However, the recall could definitely be improved.

### Decision Tree

In [61]:
kf = KFold(n_splits = 5, shuffle = True,  random_state=55)

for train, test in kf. split(Xt.index.values):
    X_train = Xt.iloc[train]
    y_train = y.iloc[train]
    X_test = Xt.iloc[test]
    y_test = y.iloc[test]

    tree = DecisionTreeClassifier(max_depth = 10)
    tree.fit(X_train,y_train)

    pred_train = tree.predict(X_train)
    pred = tree.predict(X_test)
    
    accuracy_train = []
    precision_train = []
    ra_train = []
    recall_train = []
    
    accuracy = metrics.accuracy_score(y_train, pred_train)
    accuracy_train.append(accuracy)
    precision = metrics.precision_score(y_train, pred_train)
    precision_train.append(precision)
    ra= metrics.roc_auc_score(y_train, pred_train)
    ra_train.append(ra)
    recall= metrics.recall_score(y_train, pred_train)
    recall_train.append(recall)
    
    accuracy_test = []
    precision_test = []
    ra_test = []
    recall_test = []

    accuracy = metrics.accuracy_score(y_test, pred)
    accuracy_test.append(accuracy)
    precision = metrics.precision_score(y_test, pred)
    precision_test.append(precision)
    ra= metrics.roc_auc_score(y_test, pred)
    ra_test.append(ra)
    recall= metrics.recall_score(y_test, pred)
    recall_test.append(recall)

In [62]:
printmd('**TRAIN**')
print(' Accuracy:',np.mean(accuracy_train),'\n', 'Precision:', np.mean(precision_train), 
      '\n', 'Roc Auc score:', np.mean(ra_train), '\n', 'Recall:',np.mean(recall_train))

printmd('**TEST**')
print(' Accuracy:',np.mean(accuracy_test),'\n', 'Precision:', np.mean(precision_test), 
      '\n', 'Roc Auc score:', np.mean(ra_test), '\n', 'Recall:',np.mean(recall_test))

In [63]:
dt1 = {'TRAIN Accuracy': np.mean(accuracy_train), 'TRAIN Precision': np.mean(precision_train), 
         'TRAIN ROC AUC': np.mean(ra_train), 'TRAIN Recall': np.mean(recall_train)}
dt2 = {'TEST Accuracy': np.mean(accuracy_test), "TEST Precision": np.mean(precision_test), 
        'Test ROC AUC': np.mean(ra_test), 'TEST Recall': np.mean(recall_test)}

The result of the model are no satisfactory, as precision is below 50%. The results between the train and test data are far apart in the measures, with low accuracy oo the test data.



In [64]:
roc_auc = roc_auc_score(y_test, tree.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, tree.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Decision tree (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [65]:
test_AUC.update( {'Decision tree': np.mean(ra_test)} )
train_AUC.update( {'Decision tree': np.mean(ra_train)} )
test_acc.update( {'Decision tree': np.mean(accuracy_test)} )
train_acc.update( {'Decision tree': np.mean(accuracy_train)} )

The curves overlap, with the cut-off point at best being around 55% for TP rate and 40% for FP rate. The results are highly unsatisfactory.

## Naive Bayes

In [66]:
kf = KFold(n_splits = 5, shuffle = True,  random_state=55)

for train, test in kf. split(Xt.index.values):
    X_train = Xt.iloc[train]
    y_train = y.iloc[train]
    X_test = Xt.iloc[test]
    y_test = y.iloc[test]

    nb = GaussianNB()
    nb.fit(X_train,y_train)

    pred_train = nb.predict(X_train)
    pred = nb.predict(X_test)
    
    accuracy_train = []
    precision_train = []
    ra_train = []
    recall_train = []
    
    accuracy = metrics.accuracy_score(y_train, pred_train)
    accuracy_train.append(accuracy)
    precision = metrics.precision_score(y_train, pred_train)
    precision_train.append(precision)
    ra= metrics.roc_auc_score(y_train, pred_train)
    ra_train.append(ra)
    recall= metrics.recall_score(y_train, pred_train)
    recall_train.append(recall)
    
    accuracy_test = []
    precision_test = []
    ra_test = []
    recall_test = []

    accuracy = metrics.accuracy_score(y_test, pred)
    accuracy_test.append(accuracy)
    precision = metrics.precision_score(y_test, pred)
    precision_test.append(precision)
    ra= metrics.roc_auc_score(y_test, pred)
    ra_test.append(ra)
    recall= metrics.recall_score(y_test, pred)
    recall_test.append(recall)
    
printmd('**TRAIN**')
print(' Accuracy:',np.mean(accuracy_train),'\n', 'Precision:', np.mean(precision_train), 
      '\n', 'Roc Auc score:', np.mean(ra_train), '\n', 'Recall:',np.mean(recall_train))

printmd('**TEST**')
print(' Accuracy:',np.mean(accuracy_test),'\n', 'Precision:', np.mean(precision_test), 
      '\n', 'Roc Auc score:', np.mean(ra_test), '\n', 'Recall:',np.mean(recall_test))

In [67]:
nb1 = {'TRAIN Accuracy': np.mean(accuracy_train), 'TRAIN Precision': np.mean(precision_train), 
         'TRAIN ROC AUC': np.mean(ra_train), 'TRAIN Recall': np.mean(recall_train)}
nb2 = {'TEST Accuracy': np.mean(accuracy_test), "TEST Precision": np.mean(precision_test), 
        'Test ROC AUC': np.mean(ra_test), 'TEST Recall': np.mean(recall_test)}

In [68]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, pred)
plt.figure(figsize = (8,6))
sns.heatmap(confusion_matrix, annot=True, fmt="d")

In [69]:
roc_auc = roc_auc_score(y_test, nb.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, nb.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Naive Bayes (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

Naive Bayes performed the best so far, however, our scores could also be improved.

In [70]:
test_AUC.update( {'Naive Bayes': np.mean(ra_test)} )
train_AUC.update( {'Naive Bayes': np.mean(ra_train)} )
test_acc.update( {'Naive Bayes': np.mean(accuracy_test)} )
train_acc.update( {'Naive Bayes': np.mean(accuracy_train)} )

### kNN Classifier

In [71]:
kf = KFold(n_splits = 5, shuffle = True,  random_state=55)

for train, test in kf. split(Xn.index.values):
    X_train = Xn.iloc[train]
    y_train = y.iloc[train]
    X_test = Xn.iloc[test]
    y_test = y.iloc[test]

    clf = KNeighborsClassifier(n_neighbors=25)
    clf.fit(X_train,y_train)

    pred_train = clf.predict(X_train)
    pred = clf.predict(X_test)
    
    accuracy_train = []
    precision_train = []
    ra_train = []
    recall_train = []
    
    accuracy = metrics.accuracy_score(y_train, pred_train)
    accuracy_train.append(accuracy)
    precision = metrics.precision_score(y_train, pred_train)
    precision_train.append(precision)
    ra= metrics.roc_auc_score(y_train, pred_train)
    ra_train.append(ra)
    recall= metrics.recall_score(y_train, pred_train)
    recall_train.append(recall)
    
    accuracy_test = []
    precision_test = []
    ra_test = []
    recall_test = []

    accuracy = metrics.accuracy_score(y_test, pred)
    accuracy_test.append(accuracy)
    precision = metrics.precision_score(y_test, pred)
    precision_test.append(precision)
    ra= metrics.roc_auc_score(y_test, pred)
    ra_test.append(ra)
    recall= metrics.recall_score(y_test, pred)
    recall_test.append(recall)
    
printmd('**TRAIN**')
print(' Accuracy:',np.mean(accuracy_train),'\n', 'Precision:', np.mean(precision_train), 
      '\n', 'Roc Auc score:', np.mean(ra_train), '\n', 'Recall:',np.mean(recall_train))

printmd('**TEST**')
print(' Accuracy:',np.mean(accuracy_test),'\n', 'Precision:', np.mean(precision_test), 
      '\n', 'Roc Auc score:', np.mean(ra_test), '\n', 'Recall:',np.mean(recall_test))

In [72]:
knn1 = {'TRAIN Accuracy': np.mean(accuracy_train), 'TRAIN Precision': np.mean(precision_train), 
         'TRAIN ROC AUC': np.mean(ra_train), 'TRAIN Recall': np.mean(recall_train)}
knn2 = {'TEST Accuracy': np.mean(accuracy_test), "TEST Precision": np.mean(precision_test), 
        'Test ROC AUC': np.mean(ra_test), 'TEST Recall': np.mean(recall_test)}

In [73]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, pred)
plt.figure(figsize = (8,6))
sns.heatmap(confusion_matrix, annot=True, fmt="d")

In [74]:
roc_auc = roc_auc_score(y_test, clf.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, clf.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='kNN (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

The model performs similar to Logistic Regression and Naive Bayes. However, overall our models have low score of accuracy and precision.

In [75]:
test_AUC.update( {'kNN': np.mean(ra_test)} )
train_AUC.update( {'kNN': np.mean(ra_train)} )
test_acc.update( {'kNN': np.mean(accuracy_test)} )
train_acc.update( {'kNN': np.mean(accuracy_train)} )

### Random forest

In [76]:
for train, test in kf. split(Xt.index.values):
    X_train = Xt.iloc[train]
    y_train = y.iloc[train]
    X_test = Xt.iloc[test]
    y_test = y.iloc[test]

    rf = RandomForestClassifier(n_estimators=200, max_depth=10,random_state=55,n_jobs = -1)
    rf.fit(X_train,y_train)

    pred_train = rf.predict(X_train)
    pred = rf.predict(X_test)
    
    accuracy_train = []
    precision_train = []
    ra_train = []
    recall_train = []
    
    accuracy = metrics.accuracy_score(y_train, pred_train)
    accuracy_train.append(accuracy)
    precision = metrics.precision_score(y_train, pred_train)
    precision_train.append(precision)
    ra= metrics.roc_auc_score(y_train, pred_train)
    ra_train.append(ra)
    recall= metrics.recall_score(y_train, pred_train)
    recall_train.append(recall)
    
    accuracy_test = []
    precision_test = []
    ra_test = []
    recall_test = []

    accuracy = metrics.accuracy_score(y_test, pred)
    accuracy_test.append(accuracy)
    precision = metrics.precision_score(y_test, pred)
    precision_test.append(precision)
    ra= metrics.roc_auc_score(y_test, pred)
    ra_test.append(ra)
    recall= metrics.recall_score(y_test, pred)
    recall_test.append(recall)
    
printmd('**TRAIN**')
print(' Accuracy:',np.mean(accuracy_train),'\n', 'Precision:', np.mean(precision_train), 
      '\n', 'Roc Auc score:', np.mean(ra_train), '\n', 'Recall:',np.mean(recall_train))

printmd('**TEST**')
print(' Accuracy:',np.mean(accuracy_test),'\n', 'Precision:', np.mean(precision_test), 
      '\n', 'Roc Auc score:', np.mean(ra_test), '\n', 'Recall:',np.mean(recall_test))

In [77]:
rf1 = {'TRAIN Accuracy': np.mean(accuracy_train), 'TRAIN Precision': np.mean(precision_train), 
         'TRAIN ROC AUC': np.mean(ra_train), 'TRAIN Recall': np.mean(recall_train)}
rf2 = {'TEST Accuracy': np.mean(accuracy_test), "TEST Precision": np.mean(precision_test), 
        'Test ROC AUC': np.mean(ra_test), 'TEST Recall': np.mean(recall_test)}

In [78]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, pred)
plt.figure(figsize = (8,6))
sns.heatmap(confusion_matrix, annot=True, fmt="d")

The model performs similarly to the ones above. The area under the curve suggests the cut-off point at around 60% for TP rate and less than 30% for FP rate, which could be promising.

In [79]:
logit_roc_auc = roc_auc_score(y_test, rf.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, rf.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Random forest (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [80]:
test_AUC.update( {'Random forest': np.mean(ra_test)} )
train_AUC.update( {'Random forest': np.mean(ra_train)} )
test_acc.update( {'Random forest': np.mean(accuracy_test)} )
train_acc.update( {'Random forest': np.mean(accuracy_train)} )

## Summary of models so far

In [81]:
figure(num=None, figsize=(8, 5), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(1, 2, 1)
Xlen = np.arange(len(test_AUC))
ax = plt.subplot(111)
ax.bar(Xlen, test_AUC.values(), width=0.4, color='black', align='center')
ax.bar(Xlen-0.4, train_AUC.values(), width=0.4, color='grey', align='center')
ax.legend(('Test','Train'))
plt.xticks(Xlen, test_AUC.keys())
plt.title("AUC", fontsize=17)
plt.show()

figure(num=None, figsize=(8, 5), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(1, 2, 2)
Xlen = np.arange(len(test_acc))
ax = plt.subplot(111)
ax.bar(Xlen, test_acc.values(), width=0.4, color='black', align='center')
ax.bar(Xlen-0.4, train_acc.values(), width=0.4, color='grey', align='center')
ax.legend(('Test','Train'))
plt.xticks(Xlen, test_acc.keys())
plt.title("Accuracy", fontsize=17)
plt.show()
plt.tight_layout()

From the graph, we can see that the Naive Bayes (our est data has better accuracy than train data) and Random Forest perform the best, however Random Forest had precision close to 1 on the train data and quite low score on the test data. We will try to perform the models with GridSearchCV and see whether they improve.

## Models with GridSearchCV

### Logistic regression

In [82]:
params={"penalty":["l1","l2"]}# l1 lasso l2 ridge
model = LogisticRegression(random_state=42)
grid = RandomizedSearchCV(model,params, cv = 5)
grid.fit(X_train, y_train)
print("tuned hpyerparameters :(best parameters) ",grid.best_params_)
print("accuracy :",grid.best_score_)
pred_train = grid.predict(X_train)
pred = grid.predict(X_test) 

In [83]:
printmd('**TRAIN**')
print(' Accuracy:',np.mean(metrics.accuracy_score(y_train, pred_train)),'\n', 'Precision:', np.mean(metrics.precision_score(y_train, pred_train)), 
      '\n', 'Roc Auc score:', np.mean(metrics.roc_auc_score(y_train, pred_train)), '\n', 'Recall:', np.mean(metrics.recall_score(y_train, pred_train)))

printmd('**TEST**')
print(' Accuracy:',np.mean(metrics.accuracy_score(y_test, pred)),'\n', 'Precision:', np.mean(metrics.precision_score(y_test, pred)), 
      '\n', 'Roc Auc score:', np.mean(metrics.roc_auc_score(y_test, pred)), '\n', 'Recall:',np.mean(metrics.recall_score(y_test, pred)))

In [84]:
lr3 = {'TRAIN Accuracy': np.mean(metrics.accuracy_score(y_train, pred_train)), 'TRAIN Precision': np.mean(metrics.precision_score(y_train, pred_train)), 
         'TRAIN ROC AUC': np.mean(metrics.roc_auc_score(y_train, pred_train)), 'TRAIN Recall': np.mean(metrics.recall_score(y_train, pred_train))}
lr4 = {'TEST Accuracy': np.mean(metrics.accuracy_score(y_test, pred)), "TEST Precision": np.mean(metrics.precision_score(y_test, pred)), 
        'Test ROC AUC': np.mean(metrics.roc_auc_score(y_test, pred)), 'TEST Recall': np.mean(metrics.recall_score(y_test, pred))}

In [85]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, pred)
plt.figure(figsize = (8,6))
sns.heatmap(confusion_matrix, annot=True, fmt="d")

Again, we get quite low scores (however, above 0.5) for Logistic Regression.

### Decision tree

In [86]:
params = {
    'max_depth' : [5,10,15,20]    
    }
model = DecisionTreeClassifier()
grid = RandomizedSearchCV(model,params, cv = 5)
grid.fit(X_train, y_train)
print("tuned hpyerparameters :(best parameters) ",grid.best_params_)
print("accuracy :",grid.best_score_)
pred_train = grid.predict(X_train)
pred = grid.predict(X_test) 

In [87]:
printmd('**TRAIN**')
print(' Accuracy:',np.mean(metrics.accuracy_score(y_train, pred_train)),'\n', 'Precision:', np.mean(metrics.precision_score(y_train, pred_train)), 
      '\n', 'Roc Auc score:', np.mean(metrics.roc_auc_score(y_train, pred_train)), '\n', 'Recall:', np.mean(metrics.recall_score(y_train, pred_train)))

printmd('**TEST**')
print(' Accuracy:',np.mean(metrics.accuracy_score(y_test, pred)),'\n', 'Precision:', np.mean(metrics.precision_score(y_test, pred)), 
      '\n', 'Roc Auc score:', np.mean(metrics.roc_auc_score(y_test, pred)), '\n', 'Recall:',np.mean(metrics.recall_score(y_test, pred)))

In [88]:
dt3 = {'TRAIN Accuracy': np.mean(metrics.accuracy_score(y_train, pred_train)), 'TRAIN Precision': np.mean(metrics.precision_score(y_train, pred_train)), 
         'TRAIN ROC AUC': np.mean(metrics.roc_auc_score(y_train, pred_train)), 'TRAIN Recall': np.mean(metrics.recall_score(y_train, pred_train))}
dt4 = {'TEST Accuracy': np.mean(metrics.accuracy_score(y_test, pred)), "TEST Precision": np.mean(metrics.precision_score(y_test, pred)), 
        'Test ROC AUC': np.mean(metrics.roc_auc_score(y_test, pred)), 'TEST Recall': np.mean(metrics.recall_score(y_test, pred))}

In [89]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, pred)
plt.figure(figsize = (8,6))
sns.heatmap(confusion_matrix, annot=True, fmt="d")

The Decision Tree model predicts 0s much better, however, it performs bad at predicting 1s.

### Random forest

In [90]:
kf = KFold(n_splits = 5, shuffle = True,  random_state=100000)
param_grid = { 
    'n_estimators': [100, 200, 300],
    'max_depth' : [5,10,15]
}
model = RandomForestClassifier(random_state=42)
grid = RandomizedSearchCV(model,param_grid, cv = 5)
grid.fit(X_train, y_train)
print("tuned hpyerparameters :(best parameters) ",grid.best_params_)
print("accuracy :",grid.best_score_)
pred_train = grid.predict(X_train)
pred = grid.predict(X_test) 

In [91]:
printmd('**TRAIN**')
print(' Accuracy:',np.mean(metrics.accuracy_score(y_train, pred_train)),'\n', 'Precision:', np.mean(metrics.precision_score(y_train, pred_train)), 
      '\n', 'Roc Auc score:', np.mean(metrics.roc_auc_score(y_train, pred_train)), '\n', 'Recall:', np.mean(metrics.recall_score(y_train, pred_train)))

printmd('**TEST**')
print(' Accuracy:',np.mean(metrics.accuracy_score(y_test, pred)),'\n', 'Precision:', np.mean(metrics.precision_score(y_test, pred)), 
      '\n', 'Roc Auc score:', np.mean(metrics.roc_auc_score(y_test, pred)), '\n', 'Recall:',np.mean(metrics.recall_score(y_test, pred)))

The results on the train test are incredible for the Random Forest model,however, the scores for test data compare quite poorly compared to them. However, it is still one of our best models yet.

In [92]:
rf3 = {'TRAIN Accuracy': np.mean(metrics.accuracy_score(y_train, pred_train)), 'TRAIN Precision': np.mean(metrics.precision_score(y_train, pred_train)), 
         'TRAIN ROC AUC': np.mean(metrics.roc_auc_score(y_train, pred_train)), 'TRAIN Recall': np.mean(metrics.recall_score(y_train, pred_train))}
rf4 = {'TEST Accuracy': np.mean(metrics.accuracy_score(y_test, pred)), "TEST Precision": np.mean(metrics.precision_score(y_test, pred)), 
        'Test ROC AUC': np.mean(metrics.roc_auc_score(y_test, pred)), 'TEST Recall': np.mean(metrics.recall_score(y_test, pred))}

In [93]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, pred)
plt.figure(figsize = (8,6))
sns.heatmap(confusion_matrix, annot=True, fmt="d")

In [94]:
import pickle
filename = 'Comparison_statistics.pkl'
with open('Comparison_statistics.pkl', 'wb') as output:
    pickle.dump(lr1, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(lr2, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(nb1, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(nb2, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(knn1, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(knn2, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(dt1, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(dt2, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(rf1, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(rf2, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(lr3, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(lr4, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(dt3, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(dt4, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(rf3, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(rf4, output, pickle.HIGHEST_PROTOCOL)


In [95]:
with open('Comparison_statistics.pkl', 'rb') as input:
    lr1 = pickle.load(input)
    lr2 = pickle.load(input)
    print("Logistic Regression")
    print(lr1) 
    print(lr2)
    nb1 = pickle.load(input)
    nb2 = pickle.load(input)
    print('Naive Bayes')
    print(nb1) 
    print(nb2)
    knn1 = pickle.load(input)
    knn2 = pickle.load(input)
    print('kNN')
    print(knn1) 
    print(knn2)
    dt1 = pickle.load(input)
    dt2 = pickle.load(input)
    print('Desicion Tree')
    print(dt1) 
    print(dt2)
    rf1 = pickle.load(input)
    rf2 = pickle.load(input)
    print('Random Forest')
    print(rf1) 
    print(rf2)
    lr3 = pickle.load(input)
    lr4 = pickle.load(input)
    print('Logistic regression with grid search')
    print(lr3) 
    print(lr4)
    dt3 = pickle.load(input)
    dt4 = pickle.load(input)
    print('Decision tree with grid search')
    print(dt3) 
    print(dt4)
    rf3 = pickle.load(input)
    rf4 = pickle.load(input)
    print('Random forest with grid search')
    print(rf3) 
    print(rf4)

GridSerachCV improved our Logistic Regression model, worsened our Decision Tree model, and improved our Random Forest model.

## Conclusion

At the end, all of our models perform similarly. The results are not highly satisfying, however, the dataset was tricky. There was a large amount of outliers to remove, and the explanatory variables were not highy correlated. At the end, for our testing, we only had around 2000 observations in our dataset. Therefore, taking that into consideration, our models perform quite well. Some deaper feature selection could probably improve our models.